In [1]:
from gru import GRULanguageModel
from rnn import RNNModel
from lstm import LSTMModel
from transformer import DecoderOnlyLM
import torch
import torch.nn as nn

In [2]:
model_path = 'best_model_RNNModel.pth'
TOKENIZER_PATH = "bpe_tokenizer.model"
TRAIN_FILE = "data/train.jsonl"
VAL_FILE = "data/test.jsonl"

In [3]:
rnn_model = RNNModel(
    vocab_size=10000,
    embedding_dim=128,
    hidden_dim=256,
    num_layers=2,
    dropout=0.2
).to("cuda")

lstm_model = LSTMModel(
    vocab_size=10000,
    embedding_dim=256,
    hidden_dim=256,
    num_layers=2,
    dropout=0.2,
).to("cuda")

transformer_model = DecoderOnlyLM(
    vocab_size=10000,
    d_model=256,
    nhead=4,
    num_layers=4,
    dropout=0.2
).to("cuda")

In [5]:
models = [rnn_model, lstm_model, transformer_model]

for model in models:
    model.load_state_dict(torch.load(f"best_models/best_model_{model._get_name()}.pth"))
    model.eval()
    print(f"Loaded {model._get_name()}")


Loaded RNNModel
Loaded LSTMModel
Loaded DecoderOnlyLM


In [11]:

prompt = "Once upon a time"
# Tokenize the prompt
import sentencepiece as spm
tokenizer = spm.SentencePieceProcessor(model_file=TOKENIZER_PATH)

In [12]:
input_ids = tokenizer.encode(prompt, out_type=int)
input_tensor = torch.tensor(input_ids, dtype=torch.long).to('mps').unsqueeze(0) # shape (1, seq_len)
input_tensor.shape

torch.Size([1, 4])

In [13]:
model.generate(tokenizer, prompt)

'they would be, killed leaving, and noxon or wish the young prince had possible. Rostv did not appear to savelooks and stood down for a long time to wait.Nness. Snya took her visitors to her'

In [17]:
re.shape

torch.Size([1])

In [18]:
re

tensor([2627], device='mps:0')

In [20]:
re.unsqueeze(0).shape

torch.Size([1, 1])